In [4]:
import sys
import os
import hydra
from config.config import Configuration

notebook_dir = os.path.dirname(os.path.abspath(os.getcwd()))
config_dir = os.path.join(notebook_dir,"tarea03-IA\\config")

sys.argv = [arg for arg in sys.argv if not arg.startswith("--")]

@hydra.main(config_path=config_dir, config_name="config", version_base=None)
def main(config: Configuration):
    print(config)


e:\tarea03-IA\config
